In [ ]:
dbutils.fs.ls("/FileStore/tables")

Out[360]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000)]

# **Imports**


In [ ]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

# **AWS Credentials**


In [ ]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [ ]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# **Mount s3 Bucket**


In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a70d64d47bd-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/pinterest"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[364]: True

In [ ]:
#dbutils.fs.refreshMounts()

In [ ]:

# unmount the bucket from the filestore
dbutils.fs.unmount("/mnt/pinterest")

/mnt/pinterest has been unmounted.
Out[337]: True

In [ ]:
display(dbutils.fs.ls("/mnt/mount_name/../.."))

In [ ]:
#list the topics stored on the mounted s3 bucket
display(dbutils.fs.ls("/mnt/pinterest/topics"))

# Read the contents of s3 Bucket into DataFrames

In [ ]:
topics=['pin', 'geo','user']

def get_data_into_dataframe(topic_suffix):
    # create path to topic files
    file_path = f"/mnt/pinterest/topics/0a70d64d47bd.{topic_suffix}/partition=0/*.json"
    # specify file type
    file_type = "json"
    # Ask Spark to infer the schema
    infer_schema = "true"
    # Read in JSONs from mounted S3 bucket to Spark dataframe
    df = spark.read.format(file_type) \
        .option("inferSchema", infer_schema) \
        .load(file_path)
    return df


for item in topics:
    # Name and dispaly dataframes
    get_df_statement = f"df_{item} = get_data_into_dataframe('{item}')"
    display_df_statement = f"display(df_{item})"
    # execute statements
    exec(get_df_statement)
    exec(display_df_statement)




  

# Clean df_pin DataFrame


In [ ]:
def add_none(dataframe, column, value):
    '''Find the value matched with value parameter in a column and replace with None'''
    dataframe = dataframe.withColumn(column, when(col(column).like(value),None).otherwise(col(column)))
    return dataframe
     

In [ ]:
# replace empty entries and entries with no relevant data in each column with Nones
# columns need to be replaced by null
columns_values_to_replace_none = {
    "description": "No description available%",
    "tag_list": "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",
    "follower_count": "User Info Error",
    "image_src": "Image src error.",
    "poster_name": "User Info Error",
    "title": "No Title Data Available"
}

# loop through the dictionary to replace column values with none
for column, value in columns_values_to_replace_none.items():
    df_pin = add_none(df_pin, column, value)
# Transformation made to "follower_count" column and cast column to integer
df_pin = df_pin.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
df_pin = df_pin.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))
df_pin = df_pin.withColumn("follower_count", col("follower_count").cast('int'))
# convert save_location column to include only the save location path
df_pin = df_pin.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))
# rename the index column to ind
df_pin = df_pin.withColumnRenamed("index", "ind")
# reorder columns
new_pin_column_order = [
    "ind",
    "unique_id",
    "title",
    "description",
    "follower_count",
    "poster_name",
    "tag_list",
    "is_image_or_video",
    "image_src",
    "save_location",
    "category"
]
df_pin = df_pin.select(new_pin_column_order)


# Clean df_geo DataFrame

In [ ]:
df_geo.show(50,truncate=False)

In [ ]:
from pyspark.sql.functions import array
#Create a new column coordinates that contains an array based on the latitude and longitude columns
df_geo = df_geo.withColumn("coordinates",array(df_geo.latitude,df_geo.longitude))

#Drop the latitude and longitude columns from the DataFrame
df_geo = df_geo.drop(*('latitude','longitude'))

#Convert the timestamp column from a string to a timestamp data type
df_geo = df_geo.withColumn("timestamp",to_timestamp("timestamp")) 

#Reorder the DataFrame columns
new_geo_column_order = [
    "ind",
    "country",
    "coordinates",
    "timestamp"
]

df_geo = df_geo.select(new_geo_column_order)
new_geo_column_order = [
    "ind",
    "country",
    "coordinates",
    "timestamp"
]

df_geo = df_geo.select(new_geo_column_order)

In [ ]:
df_geo

Out[372]: DataFrame[ind: bigint, country: string, coordinates: array<double>, timestamp: timestamp]

# Clean df_user DataFrame

In [ ]:
df_user.show(truncate=False)

In [ ]:
#Create a new column user_name that concatenates the information found in the first_name and last_name columns
df_user = df_user.withColumn("user_name", concat_ws(" " ,df_user['first_name'], df_user['last_name']))
#Drop the first_name and last_name columns from the DataFrame
df_user = df_user.drop(*('first_name','last_name'))
#Convert the date_joined column from a string to a timestamp data type
df_user = df_user.withColumn("date_joined",to_timestamp("date_joined")) 
#Reorder the DataFrame columns
new_user_column_order = [
    "ind",
    "user_name",
    "age",
    "date_joined"
]

df_user = df_user.select(new_user_column_order)




# Data Analysis



In [ ]:
#import for performing window functions
from pyspark.sql.window import Window
# join df_pin and df_geo dataframes on index
pin_geo = df_pin.join(df_geo, df_pin.ind == df_geo.ind)

### Most popular Pinterest category people post to based on their country.


In [ ]:
# create partition by country and order by category_count descending
windowsCountryPartition = Window.partitionBy("country").orderBy(col("category_count").desc())
# find the most popular category in each country
pin_geo.groupBy("country", "category") \
.agg(count("category") \
.alias("category_count")) \
.withColumn("rank", rank().over(windowsCountryPartition)) \
.filter(col("rank") == 1) \
.drop("rank") \
.show()

### Most popular category between years 2018 and 2022


In [ ]:
windowsYearPartition = Window.partitionBy("post_year").orderBy(col("category_count").desc())

In [ ]:
pin_geo.withColumn("post_year", year("timestamp"))\
    .filter((col("post_year") >= 2018) & (col("post_year") <= 2022)) \
    .groupBy("post_year", "category") \
    .agg(count("category") \
    .alias("category_count")) \
    .withColumn("rank", rank().over(windowsYearPartition)) \
    .filter(col("rank")==1) \
    .drop("rank") \
    .show()     

### For each country , the user with the most followers.

In [ ]:
pin_geo_user = pin_geo.join(df_user,df_pin.ind == df_user.ind)

In [ ]:
# create partition by country and order by follower_count descending
windowCountryPartition = Window.partitionBy("country").orderBy(col("follower_count").desc())

maximum_following_by_country = pin_geo.withColumn("rank", row_number().over(windowCountryPartition))\
    .filter(col("rank") == 1) \
    .select("country", "poster_name", "follower_count")\
    .orderBy(col("follower_count").desc())

maximum_following_by_country.show()

#country with highest number of followers
maximum_following_by_country.select('*')\
    .limit(1)\
    .show()  


### the most popular category people post to based on the age groups


In [ ]:
pin_user = df_pin.join(df_user, 'ind')
pin_user_age_group = pin_user.withColumn('age_group', expr("""case
 when age BETWEEN 18 AND 24 THEN '18-24'
 when age BETWEEN 25 AND 35 THEN '25-35'
 when age BETWEEN 36 AND 50 THEN '36-50'
 when age> 50 THEN '50+'
 END
 """))

In [ ]:
windowsAgePartition = Window.partitionBy("age_group").orderBy(col("category_count").desc())

In [ ]:
pin_user_age_group.groupBy("age_group","category")\
    .agg(count("category").alias("category_count"))\
    .withColumn("rank", rank().over(windowsAgePartition))\
    .filter(col("rank")==1)\
    .drop("rank")\
    .show()    


###  median follower count for users in different age groups

In [ ]:
pin_user_age_group \
.select("user_name", "date_joined", "age_group", "follower_count") \
.distinct() \
.groupBy("age_group") \
.agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
.orderBy("age_group") \
.show()

### Users joined each year

In [ ]:
pin_user_age_group.withColumn("post_year", year("date_joined"))\
    .filter((col("post_year") >= 2015) & (col("post_year") <= 2020)) \
    .groupBy("post_year") \
    .agg(count("post_year") \
    .alias("user_joined")) \
    .show() 

### The median follower count of users have joined between 2015 and 2020.

In [ ]:
pin_user_age_group \
.select("user_name", "date_joined", "follower_count") \
.distinct() \
.withColumn("post_year", year("date_joined")) \
.groupBy("post_year") \
.agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
.orderBy("post_year") \
.show()

### The median follower count of users that have joined between 2015 and 2020, based on which age group they are part of

In [ ]:
pin_user_age_group \
.select("user_name", "age_group", "date_joined", "follower_count") \
.distinct() \
.withColumn("post_year", year("date_joined")) \
.groupBy("post_year", "age_group") \
.agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
.orderBy("post_year", "age_group") \
.show()